### Notebook for the generation of an option file for Marker Processing ###

In [1]:
# Add directory to the marker processing libraries
import sys
sys.path.append('../')

### Configuration of the Parcel Source ###

In [2]:
from pds_widgets import parcel_data_source_widget
ps = parcel_data_source_widget()
ps

parcel_data_source_widget(children=(Box(children=(Label(value='Parcel Data sources:'), RadioButtons(layout=Lay…

### Configuration of the Time Series Sources ###

In [3]:
from dr_widgets import DataReaderTab
drt = DataReaderTab()
drt

DataReaderTab(children=(Tab(), HBox(children=(Button(description='Add', icon='Add', style=ButtonStyle(), toolt…

In [4]:
ts_options = drt.dump()

# Create the actual time series sources
import time_series_sources as tss

ts_source_factory = tss.time_serie_source_factory()
ts_sources = ts_source_factory.get_time_series_sources(ts_options)

# Build the signal/components dictionary
signal_components = {}
for ts_source in ts_sources :
    signal_components[ts_source.get_signal_type()] = ts_source.get_components()
    
signal_components

{'s2_bands': ['count',
  'B02_mean',
  'B02_std',
  'cloud_pct',
  'B03_mean',
  'B03_std',
  'B04_mean',
  'B04_std',
  'B05_mean',
  'B05_std',
  'B08_mean',
  'B08_std',
  'B11_mean',
  'B11_std',
  'utm_number'],
 'coh': ['count', 'VHc_mean', 'VHc_std', 'orbit', 'VVc_mean', 'VVc_std'],
 'bs': ['count', 'VHb_mean', 'VHb_std', 'orbit', 'VVb_mean', 'VVb_std']}

### Configuration of the Pre-Processors ###

In [5]:
from pp_widgets import processing_tab
pt = processing_tab(signal_components)
pt

processing_tab(children=(Tab(children=(processing_line(children=(HTML(value='<B>Processing Line</B>'), Text(va…

In [6]:
# Merge the signal/component dictionaries
sc_dict = {**signal_components, **(pt.get_signal_components())}
sc_dict

{'s2_bands': ['count',
  'B02_mean',
  'B02_std',
  'cloud_pct',
  'B03_mean',
  'B03_std',
  'B04_mean',
  'B04_std',
  'B05_mean',
  'B05_std',
  'B08_mean',
  'B08_std',
  'B11_mean',
  'B11_std',
  'utm_number'],
 'coh': ['count', 'VHc_mean', 'VHc_std', 'orbit', 'VVc_mean', 'VVc_std'],
 'bs': ['count', 'VHb_mean', 'VHb_std', 'orbit', 'VVb_mean', 'VVb_std'],
 'ndvi': ['ndvi_mean'],
 'b04_b08_b11_st': ['B08_mean', 'B11_mean', 'B04_mean'],
 'band_class': ['classes'],
 'coh_compo': ['coh_1_VHc_mean',
  'coh_1_VVc_mean',
  'coh_2_VHc_mean',
  'coh_2_VVc_mean'],
 'ndvi_filt': ['ndvi_mean'],
 'b04_b08_b11': ['B08_mean', 'B11_mean', 'B04_mean'],
 'coh_norm': ['norm']}

### Configuration of the Marker Detectors ###

In [7]:
from md_widgets import marker_detector_tab
md = marker_detector_tab(sc_dict)
md

marker_detector_tab(children=(Tab(), HBox(children=(Button(description='Add', icon='Add', style=ButtonStyle())…

### Configuration of the Data Displayers ###

In [8]:
import dd_widgets
import importlib
importlib.reload(dd_widgets)

dt = dd_widgets.data_displayer_tab(sc_dict)
dt

data_displayer_tab(children=(Tab(children=(base_data_displayer_widget(children=(SelectMultiple(description='Si…

In [9]:
import full_initializer
importlib.reload(full_initializer)
wi = full_initializer.w_initializer(ps, drt, pt, md, dt, "./config/main.json")
wi

w_initializer(children=(HTML(value='<B>Initialize all elements</B>'), Button(description='Initialize All', ico…

### Finally run the main script ###

In [10]:
%run ../main.py notebook=True parcel_num=10

Parcel data loaded 

Time series loaded


  0%|          | 0/10 [00:00<?, ?it/s]

http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=733112304&tstype=s2&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=733112304&tstype=c6&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=733112304&tstype=bs&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=751112805/2&tstype=s2&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=751112805/2&tstype=c6&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=751112805/2&tstype=bs&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=757111707/18&tstype=s2&scl=True&ref=True
<Response [200]>
http://cap.users.creodias.eu/query/parcelTimeSeries?aoi=cz&year=2020&pid=757111707/18&tstype=c6

### Check the Results ###

In [ ]:
import json
import os 
from IPython.display import display, Image

optionfile = open("./config/main.json")
options = json.load(optionfile)

path_name = None
if "data-displayers" in options :
    dd = options["data-displayers"]
    path_name = dd[0]["output_folder"]

prefix = "final"

png_file = []
if path_name is not None :
    png_files = [f for f in os.listdir(path_name) if f.startswith(prefix)]
    
if len(png_files) > 0 :
    png_file_name = path_name + "/" + png_files[0]
    img = Image(png_file_name)
    display(img)